# IMPORTAR AND DOWNLOAD DEPENDECIES

In [ ]:
%%capture
!pip install torchinfo
!pip install albumentations
!pip install torchmetrics
!pip install faster-coco-eval
!pip install torchmetrics[detection]

In [ ]:
!rm -rf TFG
!git clone https://github.com/Blancolote/TFG.git

Cloning into 'TFG'...
remote: Enumerating objects: 470, done.
remote: Counting objects: 100% (17/17), done.
remote: Compressing objects: 100% (12/12), done.
remote: Total 470 (delta 5), reused 13 (delta 5), pack-reused 453 (from 1)
Receiving objects: 100% (470/470), 126.73 KiB | 1.74 MiB/s, done.
Resolving deltas: 100% (233/233), done.


In [ ]:
from TFG.CustomFaster_MP_TPU.faster_rcnn import FasterRCNN as CustomFasterRCNN
from TFG.CustomFaster_MP_TPU.anchor_utils import AnchorGenerator as AnchorGenerator

In [ ]:
import torch
import torch.nn as nn
import torchvision
from torchvision.ops import box_iou
import torchvision.models as models
from torchvision.models import ResNet50_Weights
import random
import os
#os.environ["XLA_FLAGS"] = "--xla_dump_hlo_as_text --xla_dump_to=/tmp/xla"
#os.environ['XLA_IR_DEBUG'] = '1'
#os.environ['XLA_HLO_DEBUG'] = '1'
import albumentations as A
import pandas as pd
from torchvision import transforms
from tqdm import tqdm
import torch.optim as optim
from torch.cuda.amp import autocast
from torch.utils.data import Dataset, DataLoader
import cv2
import matplotlib.pyplot as plt
import numpy as np
import pickle
from pprint import pprint
from torchinfo import summary
import seaborn as sns
from dataclasses import dataclass
import torch_xla
import torch_xla.core.xla_model as xm
import torch_xla.amp.syncfree.adamw as adamw
from torch_xla.distributed.parallel_loader import MpDeviceLoader
import torch_xla.debug.metrics as met
import warnings
from collections import OrderedDict
from typing import Dict, List, Optional, Tuple, Union
from torchvision.ops.feature_pyramid_network import FeaturePyramidNetwork
from torchmetrics.detection.mean_ap import MeanAveragePrecision
from sklearn.model_selection import train_test_split
from torch.utils.data import Subset
import pycocotools
from sklearn.metrics import accuracy_score, recall_score, roc_curve, auc, confusion_matrix, f1_score
import time
import random


In [ ]:
"""ground_truth_boxes = torch.tensor([[12,12,45,45],[12,12,45,45]], dtype=torch.float32)

# Predicciones del modelo
predicted_boxes = torch.tensor([
    [12, 12, 48, 48],[12,12,45,45],   # buena predicción de la box 1
], dtype=torch.float)

# Calculamos la matriz de IoU
ious = box_iou(predicted_boxes, ground_truth_boxes)

print("Matriz IoU:")
print(ious)

mean_iou = ious.mean().item()
print("IoU promedio (por predicción):", mean_iou)"""

'ground_truth_boxes = torch.tensor([[12,12,45,45],[12,12,45,45]], dtype=torch.float32)\n\n# Predicciones del modelo\npredicted_boxes = torch.tensor([\n    [12, 12, 48, 48],[12,12,45,45],   # buena predicción de la box 1\n], dtype=torch.float)\n\n# Calculamos la matriz de IoU\nious = box_iou(predicted_boxes, ground_truth_boxes)\n\nprint("Matriz IoU:")\nprint(ious)\n\nmean_iou = ious.mean().item()\nprint("IoU promedio (por predicción):", mean_iou)'

In [ ]:
"""ious_list = [0.7, 0.65, float('nan'), 0.8]

# Para calcular el promedio ignorando los nan:
import numpy as np
mean_over_dataset = np.nanmean(ious_list)
mean_over_dataset"""

"ious_list = [0.7, 0.65, float('nan'), 0.8]\n\n# Para calcular el promedio ignorando los nan:\nimport numpy as np\nmean_over_dataset = np.nanmean(ious_list)\nmean_over_dataset"

# DATALOADER

This block defines the Dataloader used for the Faster R-CNN.


In [ ]:
def resizeAndTransform(bbox, original_size, new_size):
    """
    Transforms a bounding box from (x, y, width, height) to (x_min, y_min, x_max, y_max)
    and rescales it to a new size.
    
    Args:
    -bbox: Bounding box in the format (x, y, width, height). x and y are the top-left corner coordinates.
    - original_size: tuple(original_width, original_height) of the image.
    - new_size: tuple(new_width, new_height) new size of the image.

    Returns:
    - Resized bounding box in Pascal VOC format (x_min, y_min, x_max, y_max)
    """

    x, y, width, height = bbox

    original_width, original_height = original_size
    new_width, new_height = new_size

    x_min = x
    y_min = y
    x_max = x + width
    y_max = y + height

    x_min_resized = x_min * (new_width / original_width)
    y_min_resized = y_min * (new_height / original_height)
    x_max_resized = x_max * (new_width / original_width)
    y_max_resized = y_max * (new_height / original_height)

    x_min_resized = max(0, min(x_min_resized, new_width))
    y_min_resized = max(0, min(y_min_resized, new_height))
    x_max_resized = max(0, min(x_max_resized, new_width))
    y_max_resized = max(0, min(y_max_resized, new_height))

    return [x_min_resized, y_min_resized, x_max_resized, y_max_resized]

In [ ]:
def creator_path_slices(patient_dir):
    return [os.path.join(patient_dir,s) for s in os.listdir(patient_dir)]

In [ ]:

def get_augmentations():
  rotation = random.randint(-10,10)
  return A.Compose([A.Affine(rotate=(rotation,rotation),p=0.1),
                    A.Affine(
                        scale=(1 - 0.15, 1 + 0.15),
                        translate_percent=(-0.1, 0.1),
                        p=0.8, border_mode=cv2.BORDER_CONSTANT),
                    A.RandomBrightnessContrast(
                    brightness_limit=0.1,
                    contrast_limit=0.1,
                    p=0.5
                ),
                A.GaussianBlur(blur_limit=(3, 5), p=0.1),
                A.GaussNoise(std_range=(0.05, 0.1), p=0.2),
    ], bbox_params=A.BboxParams(format='pascal_voc', min_visibility=0.2, label_fields=['labels']))

class TomosynthesisDataset(Dataset):
  def __init__(self, csv_path, images_folder, augmentations=None):
      self.metadata = pd.read_csv(csv_path)
      self.images_folder = images_folder
      self.augmentations = augmentations
      self.data = []
      self.empty_boxes = torch.zeros((0,4), dtype=torch.bfloat16)

      for i in range(len(self.metadata)):
          patient_id = self.metadata.iloc[i]['PatientID']
          target_slice = self.metadata.iloc[i]['Slice_representativo']
          intermediate_folder = self.metadata.iloc[i]['Intermedia']
          patient_folder = os.path.join(images_folder, patient_id)
          x = self.metadata.iloc[i]['X']
          y = self.metadata.iloc[i]['Y']
          width = self.metadata.iloc[i]['Width']
          height = self.metadata.iloc[i]['Height']
          classification = self.metadata.iloc[i]['Clasificacion']
          original_width = self.metadata.iloc[i]['Original_width']
          original_height = self.metadata.iloc[i]['Original_height']

          entry = {
              'patient_id': patient_id,
              'target_slice': target_slice,
              'intermediate_folder': os.path.join(patient_folder, intermediate_folder),
              'X': x,
              'Y': y,
              'Width': width,
              'Height': height,
              'Classification': classification,
              'Original_width': original_width,
              'Original_height' : original_height
          }

          if classification == 1 and (random.random() < 0.6): #all images that have anomaly could be data augmented within 50% of posibilities
            
            self.data.append((entry, False))  # Original image
            
            self.data.append((entry, True))   # Data augmented image
          else:
            self.data.append((entry,False))



  def __len__(self):
      return len(self.data)

  def __getitem__(self, idx):
      
      entry, apply_augmentation = self.data[idx]  

      distancia = 100
      target_slice_subsample_position = 0
      intermediate_folder = entry['intermediate_folder']
      target_slice = entry['target_slice']
      x = entry['X']
      y = entry['Y']
      width = entry['Width']
      heigth = entry['Height']
      classification = entry['Classification']
      original_width = entry['Original_width']
      original_height = entry['Original_height']

      bbox_albumentationsFormat = None
      if classification == 0:
        bbox_albumentationsFormat = self.empty_boxes
      else:
        bbox_albumentationsFormat = [resizeAndTransform((x,y,width,heigth), (original_width,original_height), (256,256))]

      paths_slices = creator_path_slices(intermediate_folder)

      if len(paths_slices) != 27:
          raise ValueError(f"Expected 27 slices, got {len(paths_slices)} for patient {entry['patient_id']}")

      for i in range(len(paths_slices)):
          name_slice = paths_slices[i].split('/')[-1]
          num_slice = int(name_slice.split('.')[0])

          if abs(target_slice - num_slice) < distancia:
              target_slice_subsample_position = i
              distancia = abs(target_slice - num_slice)
              popo = num_slice

      slices = []
      transform = transforms.ToTensor()

      for path in paths_slices:
          if not os.path.exists(path):
              raise FileNotFoundError(f"Slice not found: {path}")

          slice_img = cv2.imread(path, cv2.IMREAD_COLOR)
          slice_img = cv2.cvtColor(slice_img, cv2.COLOR_BGR2RGB)
          slice_img = (slice_img / 255.0).astype(np.float32)

          # Aplicar augmentación
          if apply_augmentation==True and self.augmentations is not None:  
              transformed = self.augmentations(image=slice_img, bboxes = np.array(bbox_albumentationsFormat, dtype=np.float32), labels=[classification] )
              slice_img = transformed['image']
              transformed_bbox = transformed['bboxes']
          else:
            transformed_bbox = bbox_albumentationsFormat


          slice_tensor = transform(slice_img)
          slices.append(slice_tensor)

      slices_tensor = torch.stack(slices)

      classification = torch.tensor([classification], dtype=torch.long)  #ensure that is not an empty tensor

      if isinstance(transformed_bbox, torch.Tensor):
        if len(transformed_bbox.shape)!=2:
          transformed_bbox = transformed_bbox.unsqueeze(0)
        transformed_bbox_tensor = transformed_bbox.clone().detach().float()
      else:
        transformed_bbox = torch.tensor(transformed_bbox, dtype=torch.bfloat16)
        if len(transformed_bbox.shape)!=2:
          transformed_bbox = transformed_bbox.unsqueeze(0)
        transformed_bbox_tensor = transformed_bbox.clone().detach().float()

      transformed_bbox_tensor = torch.clamp(transformed_bbox_tensor,0,256)

      dic_labels = {"boxes": transformed_bbox_tensor, "labels": classification}
      return slices_tensor, dic_labels, torch.tensor(target_slice_subsample_position, dtype=torch.long)


In [ ]:
def collate_fn(batch):
    images, targets, positions = zip(*batch)
    structured_targets = []
    images = torch.stack(images)

    for t in targets:
        if len(t) > 0:
            # extract boxes and labels from the dictionary to ensure they are in the correct format
            boxes = t["boxes"]
            labels = t["labels"]

        structured_targets.append({
            "boxes": boxes,
            "labels": labels
        })

    return images, structured_targets, torch.tensor(positions, dtype=torch.long)

# INTERMEDIATE MODULE

Takes the input from the bakcbone of 27 slices and turns it into a one feature map for each tomosynthesis image.
Then a FPN is applied

In [ ]:
class FeatureMapMerger(nn.Module):
    def __init__(self):
        super(FeatureMapMerger, self).__init__()
        self.fpn = FeaturePyramidNetwork(
            in_channels_list=[512, 1024, 2048],
            out_channels=512     #output size of the feature map channels
        )

    def forward(self, logits, c3, c4, c5):

      assert logits.ndim == 2, "logits should have shape (batch_size, num_slices)"

      fused_features_dic = {}

      list_features = [c3, c4, c5]

      # Calc weights
      weights = torch.softmax(logits, dim=1)  # (batch_size, num_slices)

      positions = torch.argmax(logits, dim=1)

      # From (batch_size, num_slices) to (batch_size, num_slices, 1, 1, 1)
      expanded_weights = weights[:, :, None, None, None]

      for i,feature_map in enumerate(list_features):

        assert feature_map.ndim == 5, "feature maps should have shape (batch_size, num_slices, num_feature_maps, height, width)"

        # feature_maps tiene forma (batch_size, num_slices, num_feature_maps, height, width)
        weighted_feature_map = feature_map * expanded_weights

        # One feature map per image
        fused_features_dic[str(i)] = torch.sum(weighted_feature_map, dim=1)

      fpn_out = self.fpn(
          fused_features_dic
      )

      return fpn_out, positions


# BACKBONE

Backbone presented in the file backbone_training.ipynb with no FPN

In [ ]:
class RepresentativeSliceDetector(nn.Module):
    def __init__(self, feature_dim=512, hidden_dim=256, dropout=0.3):
        super(RepresentativeSliceDetector, self).__init__()
        self.feature_dim = feature_dim

        self.attention = nn.Sequential(
            nn.Linear(self.feature_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim, 1)
        )
        self.layer_norm = nn.LayerNorm(self.feature_dim)

    def forward(self, feature_map):

        features = feature_map.mean(dim=(3, 4))  

        features = self.layer_norm(features)
        logits = self.attention(features)

        return logits.squeeze(-1)

In [ ]:
class ResNet50FeatureExtractor(nn.Module):
    def __init__(self, weights=ResNet50_Weights.DEFAULT, dropout=0.3):
        super(ResNet50FeatureExtractor, self).__init__()
        self.resnet50 = models.resnet50(weights=weights)
        self.stem = nn.Sequential(
            self.resnet50.conv1,
            self.resnet50.bn1,
            self.resnet50.relu,
            self.resnet50.maxpool  
        )

        self.layer1 = self.resnet50.layer1  
        self.layer2 = self.resnet50.layer2  
        self.layer3 = self.resnet50.layer3  
        self.layer4 = self.resnet50.layer4  

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        batch_size, num_slices, C, H, W = x.size()
        x = x.view(batch_size * num_slices, C, H, W)

        x = self.stem(x)
        c2 = self.layer1(x)  
        c3 = self.layer2(c2)  
        c4 = self.layer3(c3)  
        c5 = self.layer4(c4)  

        def reshape(f): return f.view(batch_size, num_slices, f.size(1), f.size(2), f.size(3))

        return {
            'c3': reshape(c3),
            'c4': reshape(c4),
            'c5': reshape(c5)
        }


In [ ]:
class ModifiedResNet50Backbone(nn.Module): 
    def __init__(self, out_channels=2048):
        super(ModifiedResNet50Backbone, self).__init__()

        try:
          self.device = xm.xla_device()
          self.dtype = torch.bfloat16 if 'xla' in str(self.device) else torch.float32
        except:
          self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
          self.dtype = torch.float16 if "cuda" in str(self.device) else torch.bfloat16


        self.out_channels = out_channels
        self.feature_extractor = ResNet50FeatureExtractor()
        self.attention_module = RepresentativeSliceDetector(feature_dim=self.out_channels)

        self.to(self.dtype)


    def forward(self, x):
        x = x.to(self.device)

        top_indices_logits = None

        features = self.feature_extractor(x)
        c3, c4, c5 = features["c3"], features["c4"], features["c5"]

        top_indices_logits = self.attention_module(c5)

        return c3, c4, c5, top_indices_logits


In [ ]:
class CustomBackbone(nn.Module):
  def __init__(self):
    super().__init__()
    self.backbone = ModifiedResNet50Backbone()
    self.backbone.load_state_dict(torch.load("/content/drive/MyDrive/backbone.pth"))
    for param in self.backbone.parameters():
            param.requires_grad = False

    self.intermediate = FeatureMapMerger()
    self.out_channels = 512

  def forward(self, x):

    c3 ,c4 ,c5 , logits = self.backbone(x)
    fused_feature_map, positions= self.intermediate(logits, c3, c4, c5)

    representativeSlices = x[torch.arange(x.shape[0]), positions]  #most representative slice per image


    return fused_feature_map, logits, representativeSlices



# FASTER

## Outputs examples


In training:



```
Pérdidas: {'loss_classifier': tensor(0.6855, grad_fn=<NllLossBackward0>), 'loss_box_reg': tensor(9.6547e-06, grad_fn=<DivBackward0>), 'loss_objectness': tensor(0.6846, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>), 'loss_rpn_box_reg': tensor(0.1400, grad_fn=<DivBackward0>)}
Logits: tensor([[-0.4513, -0.1834,  0.0338, -0.5055, -0.5497, -0.0438,  0.2732, -0.1832,
         -0.0653,  0.0784, -0.5180,  0.0356, -0.2929, -0.2163, -0.2548,  0.0238,
         -0.1174, -0.3515,  0.2769, -0.2075, -0.1633, -0.0156, -0.3062, -0.1422,
          0.1197, -0.3001, -0.2209],
        [-0.0558, -0.0029, -0.0046, -0.0118, -0.2489, -0.1658, -0.0777,  0.1584,
         -0.0571,  0.4947,  0.0257, -0.2142, -0.0926,  0.4675, -0.0921, -0.2695,
          0.4509, -0.2853, -0.3968,  0.0142,  0.1585, -0.3831, -0.5032, -0.3430,
         -0.3413, -0.3068, -0.6169],
        [-0.2340,  0.1319, -0.0977,  0.1093, -0.4260, -0.4540,  0.0163,  0.0521,
          0.2435,  0.3576, -0.0256, -0.3542,  0.2086,  0.0764,  0.0368, -0.0441,
         -0.1628, -0.1285, -0.1255, -0.3494, -0.1908, -0.4324, -0.2591,  0.3431,
         -0.3080,  0.2426, -0.1217],
        [-0.3487, -0.0390,  0.0026,  0.0154,  0.2411, -0.1290, -0.1990, -0.7018,
         -0.1884, -0.0971, -0.1541,  0.1258, -0.3235, -0.5454, -0.2858,  0.2599,
         -0.0584, -0.1472,  0.2348, -0.1662, -0.0596,  0.0074, -0.3792,  0.2995,
         -0.0809,  0.4552, -0.1480]], grad_fn=<SqueezeBackward1>)


```



In inference:


```
Outputs: [{'boxes': tensor([[0., 0., 0., 0.]]), 'labels': tensor([0]), 'scores': tensor([1.])}, {'boxes': tensor([[0., 0., 0., 0.]]), 'labels': tensor([0]), 'scores': tensor([1.])}, {'boxes': tensor([[0., 0., 0., 0.]]), 'labels': tensor([0]), 'scores': tensor([1.])}, {'boxes': tensor([[0., 0., 0., 0.]]), 'labels': tensor([0]), 'scores': tensor([1.])}]
Logits: tensor([[-0.1839, -0.1884, -0.1937, -0.2048, -0.1715, -0.1859, -0.1893, -0.1789,
         -0.1866, -0.2029, -0.1892, -0.2091, -0.1994, -0.1818, -0.1888, -0.1641,
         -0.1866, -0.1924, -0.1870, -0.1784, -0.1886, -0.1930, -0.1782, -0.1883,
         -0.1843, -0.1954, -0.1632],
        [-0.1678, -0.1775, -0.1724, -0.1965, -0.1880, -0.1826, -0.1909, -0.1788,
         -0.1652, -0.1920, -0.2019, -0.2011, -0.2021, -0.1774, -0.1920, -0.1714,
         -0.1959, -0.1871, -0.1885, -0.1809, -0.1601, -0.1790, -0.2130, -0.1581,
         -0.1681, -0.1959, -0.1895],
        [-0.2171, -0.1849, -0.1740, -0.1858, -0.1918, -0.2019, -0.2090, -0.1816,
         -0.1881, -0.2034, -0.1840, -0.1888, -0.1935, -0.1629, -0.1875, -0.1790,
         -0.2033, -0.1882, -0.1541, -0.1910, -0.2118, -0.2116, -0.1986, -0.1918,
         -0.1800, -0.1922, -0.2059],
        [-0.1599, -0.1789, -0.1731, -0.1946, -0.1949, -0.1954, -0.2328, -0.1558,
         -0.1741, -0.2072, -0.1961, -0.2003, -0.1710, -0.1786, -0.1925, -0.1976,
         -0.1944, -0.2055, -0.2001, -0.2203, -0.2082, -0.2003, -0.1960, -0.1843,
         -0.1917, -0.1862, -0.1707]])
```



## Custom Faster

In [ ]:
class CustomFaster(nn.Module):
  """
  Modification of FasterRCNN, with a custom backbone and a intermediate module presented before.
  Is made to be trained with a TPU and can distinguish between two classes: background and anomaly. When inference,
  the model will select as an anomaly does outputs with more that a 0.5 score and those which boxes is
  composed of less than 95% of dark pixels. For more information, see the source code.
  """

  def __init__(self):

    super().__init__()
    self.backbone = CustomBackbone()
    try:
      self.device = xm.xla_device()
      self.dtype = torch.bfloat16 if 'xla' in str(self.device) else torch.float32

    except:
      if torch.cuda.is_available():
        self.device = torch.device("cuda")
        self.dtype = torch.float16
      else:
        self.device = torch.device("cpu")
        self.dtype = torch.float32

    anchor_generator = AnchorGenerator(
      sizes = ((8, 16, 256), (32, 48, 256), (64, 96, 256)), #256 is for actionable cases
      aspect_ratios=((0.5, 1.0, 1.5),(0.5, 1.0, 1.5),(0.5, 1.0, 1.5))
    )

    roi_pooler = torchvision.ops.MultiScaleRoIAlign(
      featmap_names=['0', '1', '2'], output_size=5, sampling_ratio=2
    )

    self.fasterRCNN = CustomFasterRCNN(
        backbone=self.backbone,
        num_classes=2,
        rpn_anchor_generator=anchor_generator,
        box_roi_pool=roi_pooler,
      )

  def forward(self, slice_tensor, targets=None):
    """
    slices_tensor: Slices tensor of size (batch_size, num_slices, C, H, W).
    targets: Dic with 'boxes' and 'labels' (only used when training).
    """

    slice_tensor = slice_tensor.to(self.dtype)
    if self.training:
      if targets is None or not isinstance(targets, list) or not all(isinstance(t, dict) for t in targets):
        raise ValueError(f"`Targets` should be a dic list, but {type(targets)} was passed")
      targets = [{k: v.to(self.dtype) if torch.is_tensor(v) and k != "labels"
                           else v for k, v in t.items()} for t in targets]
      losses, logits =  self.fasterRCNN(slice_tensor,targets )
      return losses, logits

    else:

      outputs, logits = self.fasterRCNN(slice_tensor)

      threshold_black_ratio = 0.90
      pixel_threshold = 0.1

      for i, output in enumerate(outputs):
          boxes = output['boxes']
          scores = output['scores']
          labels = output['labels']

          if (labels == 1).any():
              anomaly_mask = (labels == 1)
              anomaly_scores = scores[anomaly_mask]
              anomaly_boxes = boxes[anomaly_mask]

              best_idx = torch.argmax(anomaly_scores)
              best_box = anomaly_boxes[best_idx].int()
              x1, y1, x2, y2 = best_box.tolist()

              cropped = slice_tensor[i, :, y1:y2, x1:x2]
              black_pixel_ratio = (cropped < pixel_threshold).float().mean()

              if black_pixel_ratio >= threshold_black_ratio:
                  outputs[i] = {
                      'boxes': torch.zeros((0, 4), device=self.device),
                      'scores': torch.zeros((0), device=self.device),
                      'labels': torch.zeros((0), device=self.device, dtype=torch.long)
                  }



      return outputs,logits



# Load Dataloader

## Load data from a split Dataset


In [ ]:
train_dataset = TomosynthesisDataset(
      csv_path= "csv_train_path",
      images_folder='image_folder_train_path',
      augmentations = get_augmentations()
    )

val_dataset = TomosynthesisDataset(
  csv_path='csv_test_path',
  images_folder='image_folder_val_path',
  )

train_dataloader = DataLoader(
    train_dataset,
    batch_size=8,
    shuffle=True,
    pin_memory=True,
    num_workers=2,
    drop_last=True,
    collate_fn=collate_fn
  )


val_dataloader = DataLoader(
    val_dataset,
    batch_size=8,
    shuffle=False,
    pin_memory=True,
    num_workers=2,
    drop_last=True,
    collate_fn=collate_fn
  )


## Load data from 1 big dataset


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/slices_test+valid.csv')
labels = df['Clasificacion'].tolist()
indices = list(range(len(df)))

train_idx, val_idx = train_test_split(indices, test_size=0.2, stratify=labels, random_state=42)

half_train_size = len(train_idx) // 3
half_val_size = len(val_idx) // 3

train_idx_small = train_idx[:half_train_size]
val_idx_small = val_idx[:half_val_size]

base_dataset = TomosynthesisDataset(
    csv_path='csv_path',
    images_folder='image_folder_path',
    augmentations=None  
)

train_dataset = Subset(TomosynthesisDataset(
    csv_path='csv_path',
    images_folder='/content/drive/MyDrive/test+valid_png',
    augmentations=get_augmentations()
), train_idx)

val_dataset = Subset(TomosynthesisDataset(
    csv_path='csv_path',
    images_folder='image_folder_path',
    augmentations=None
), val_idx)

train_dataloader = DataLoader(
    train_dataset,
    batch_size=8,
    shuffle=True,
    pin_memory=True,
    num_workers=2,
    collate_fn=collate_fn,
    persistent_workers=True
)

val_dataloader = DataLoader(
    val_dataset,
    batch_size=8,
    shuffle=False,
    pin_memory=True,
    num_workers=2,
    collate_fn=collate_fn,
    persistent_workers=True
)

print(f'Tamaño train batches: {len(train_dataset)}.')
print(f'Tamaño val batches: {len(val_dataset)}.')
train_labels = [df.iloc[idx]['Clasificacion'] for idx in train_idx]
train_class_counts = pd.Series(train_labels).value_counts().to_dict()
print("Distribución de clases en train_dataset:")
for class_name, count in train_class_counts.items():
    print(f"  - {class_name}: {count} imágenes")

val_labels = [df.iloc[idx]['Clasificacion'] for idx in val_idx]
val_class_counts = pd.Series(val_labels).value_counts().to_dict()
print("\nDistribución de clases en val_dataset:")
for class_name, count in val_class_counts.items():
    print(f"  - {class_name}: {count} imágenes")

Tamaño train batches: 768.
Tamaño val batches: 192.
Distribución de clases en train_dataset:
  - 0: 1819 imágenes
  - 1: 486 imágenes

Distribución de clases en val_dataset:
  - 0: 455 imágenes
  - 1: 122 imágenes


## Funciones entrenamiento complementarias

In [ ]:
def warmup_model(model, device):
  """ Warm up the model to avoid slow first iteration and compile XLA graphs.
  It is used one batch of 8 images, half of them with anomaly and half without it."""

  print("Performing XLA warm up...")

  batch_size = 8
  dummy_images = torch.rand(batch_size, 27, 3, 256, 256, device=device)


  dummy_labels = []

  for i in range(batch_size // 4):
      dummy_labels.append({
          'boxes': torch.tensor([[10, 10, 50, 50]], device=device),
          'labels': torch.tensor([1], device=device)
      })


  for i in range(batch_size // 4, batch_size):
      dummy_labels.append({
          'boxes': torch.empty(((0,4)), dtype=torch.float32),
          'labels': torch.tensor([0], device=device)
      })


  model.train()
  with torch.no_grad():
      with torch_xla.amp.autocast(device=device, dtype=torch.bfloat16):
          output, logits = model(dummy_images, dummy_labels)

          loss = sum(output.values())

  model.eval()
  with torch.no_grad():
      with torch_xla.amp.autocast(device=device, dtype=torch.bfloat16):
          output, logits = model(dummy_images)

  xm.mark_step()
  print("Warm up complete.")

# Fucntion to reset metrics buffer 
def reset_metrics_buffer(buffer):
  for key in buffer:
      buffer[key] = 0.0

In [ ]:
def reduceOutputs(gt_box, outputBoxes, device = xm.xla_device):

  """Function to reduce the outputs of the model. It is used to calculate the IoU between the ground truth boxes and the output boxes.
  There are four cases:
  1. No ground truth boxes and no output boxes: perfect IoU for negative cases.
  2. No ground truth boxes and output boxes: false positives.
  3. Ground truth boxes and no output boxes: false negatives.
  4. Ground truth boxes and output boxes: potential true positives."""

  if gt_box.size(0) == 0:

    if outputBoxes.size(0) == 0:

      return torch.tensor(1.0, device=device), -1 #perfect IoU for negative cases

    return torch.tensor(0.0, device=device), -1 #false positives

  if outputBoxes.size(0) == 0:
    return torch.tensor(0.0, device=device), -1 #false negatives

  ious = box_iou(gt_box, outputBoxes) #potential true positives
  best_iou, position = torch.max(ious.squeeze(0), dim=0)

  return best_iou, position.item()


## Función entrenamiento y validación

In [ ]:
def trainAndValid(model, train_loader, val_loader, num_epochs=1, lr=1e-4):
    device = xm.xla_device()
    model = model.to(device)

    warmup_model(model, device)

    train_loader = MpDeviceLoader(train_loader, device)
    val_loader = MpDeviceLoader(val_loader, device)

    optimizer = adamw.AdamW(model.parameters(), lr=lr, weight_decay=1e-5)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, mode='max', factor=0.5, patience=4
    )

    best_combined_score = 0
    patience_earlyStop = 10
    epochs_no_improve = 0


    dummy_box = torch.tensor([[0, 0, 1, 1]], device=device)  # dummy box to maintain consistency
    background_label = torch.tensor([0], device=device) # dummy label for background

    # Pre-alocate buffers for metrics (to avoid frequent CPU-TPU transfers)
    train_metrics_buffer = {
        "loss": 0.0,
        "loss_faster": 0.0,
        "correct_attention": 0,
        "samples": 0
    }

    training_history = {
        "train_loss": [], "train_acc_attention": [],
        "train_preds": [], "train_mri": [], "val_acc_attention": [],
        "val_acc_faster_class": [], "val_labels_faster_class": [],
        "val_preds_faster_class": [], "val_labels_faster_boxes": [],
        "val_preds_faster_boxes": [],  "val_iou": [],
    }


    for epoch in range(num_epochs):

        # ===== TRAINING PHASE =====

        model.train()
        reset_metrics_buffer(train_metrics_buffer)
        train_preds = torch.tensor([], device=device)
        train_mri = torch.tensor([], device=device)

        train_loader_tqdm = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs} [Training]", leave=True)

        for images, labels, mri in train_loader_tqdm:
            images = images.to(device)
            labels_device = [{k: v.to(device) for k, v in label.items()} for label in labels]
            mri = mri.to(device)

            with torch_xla.amp.autocast(device=device, dtype=torch.bfloat16):

                output, logits = model(images, labels_device)
                loss = sum(output.values())

            loss.backward()

            batch_size = images.size(0)
            with torch.no_grad():
                train_metrics_buffer["loss"] += loss.detach() * batch_size

                preds_attention = torch.argmax(logits, dim=1)
                train_metrics_buffer["correct_attention"] += (abs(preds_attention - mri) <= 2).sum()
                train_metrics_buffer["samples"] += batch_size

            train_loader_tqdm.set_postfix(loss=loss.detach())

            train_preds = torch.cat([train_preds, preds_attention], dim=0)
            train_mri = torch.cat([train_mri, mri], dim=0)

            xm.optimizer_step(optimizer)
            optimizer.zero_grad()

        train_loss = train_metrics_buffer["loss"] / train_metrics_buffer["samples"]
        train_acc_attention = train_metrics_buffer["correct_attention"] / train_metrics_buffer["samples"]

        training_history["train_loss"].append(train_loss)
        training_history["train_acc_attention"].append(train_acc_attention)
        training_history["train_preds"].append(train_preds.cpu().numpy())
        training_history["train_mri"].append(train_mri.cpu().numpy())

      # ===== VALIDATION PHASE =====
        model.eval()

        # Pre-alocate buffers for validation metrics
        val_metrics = {
            "correct_attention": 0,
            "correct_faster_class": 0,
            "samples": 0,
            "iou": 0.0
        }

        val_results = {
          "preds_faster_class": torch.tensor([], device=device),
          "labels_faster_class": torch.tensor([], device=device),
          "preds_faster_boxes": torch.tensor([], device=device),
          "labels_faster_boxes": torch.tensor([], device=device),
        }

        with torch.no_grad():
            val_loader_tqdm = tqdm(val_loader, desc=f"Epoch {epoch+1}/{num_epochs} [Validation]", leave=True)

            
            for images, labels, mri in val_loader_tqdm:
                images = images.to(device)
                labels_device = [{k: v.to(device) for k, v in label.items()} for label in labels]
                mri = mri.to(device)

                with torch_xla.amp.autocast(device=device, dtype=torch.bfloat16):
                    output, logits = model(images)

                
                preds_attention = torch.argmax(logits, dim=1)
                val_metrics["correct_attention"] += (abs(preds_attention - mri) <= 2).sum()
                val_metrics["samples"] += mri.size(0)

                for i in range(len(labels)):

                  iou, index = reduceOutputs(labels[i]['boxes'], output[i]['boxes'], device)

                  if index == -1:
                    # No ground truth boxes or no output boxes, or both
                    scoreBased_index = torch.argmax(output[i]['scores'], dim=0)

                    gt_box = dummy_box if labels[i]['boxes'].size(0) == 0 else labels[i]['boxes']
                    if output[i]['boxes'].size(0) == 0:
                      pred_box = dummy_box
                      pred_label = background_label
                    else:
                      pred_box = output[i]['boxes'][scoreBased_index]
                      pred_label = output[i]['labels'][scoreBased_index].unsqueeze(0)
                    

                    val_metrics["correct_faster_class"] += (labels[i]['labels'] == pred_label)

                    val_results["preds_faster_class"] = torch.cat([val_results["preds_faster_class"],
                                                                  pred_label])
                    val_results["labels_faster_class"] = torch.cat([val_results["labels_faster_class"],
                                                                  labels[i]['labels']])


                    val_results["labels_faster_boxes"] = torch.cat([val_results["labels_faster_boxes"], gt_box])
                    val_results["preds_faster_boxes"] = torch.cat([val_results["preds_faster_boxes"], pred_box])

                    val_metrics["iou"] += iou  

                  else: #normal case, with gt boxes and pred boxes
                    
                    box = output[i]['boxes'][index].unsqueeze(0)
                    label = output[i]['labels'][index].unsqueeze(0)


                    val_metrics["correct_faster_class"] += (label == labels[i]["labels"]).sum().item()
                    val_results["preds_faster_class"] = torch.cat([val_results["preds_faster_class"], label])
                    val_results["labels_faster_class"] = torch.cat([val_results["labels_faster_class"], labels[i]['labels']])
                    val_results["labels_faster_boxes"] = torch.cat([val_results["labels_faster_boxes"], labels[i]['boxes']])
                    val_results["preds_faster_boxes"] = torch.cat([val_results["preds_faster_boxes"], box])
                    val_metrics["iou"] += iou

            val_loader_tqdm.set_postfix()

        val_acc_attention = val_metrics["correct_attention"] / val_metrics["samples"]
        val_acc_faster_class = val_metrics["correct_faster_class"] / val_metrics["samples"]
        val_iou = val_metrics["iou"] / val_metrics["samples"]


        training_history["val_acc_attention"].append(val_acc_attention)
        training_history["val_acc_faster_class"].append(val_acc_faster_class)
        training_history["val_iou"].append(val_iou)

        training_history["val_labels_faster_class"].append(val_results["labels_faster_class"].cpu().numpy())
        training_history["val_preds_faster_class"].append(val_results["preds_faster_class"].cpu().numpy())
        training_history["val_labels_faster_boxes"].append(val_results["labels_faster_boxes"].cpu().numpy())
        training_history["val_preds_faster_boxes"].append(val_results["preds_faster_boxes"].cpu().numpy())

        combined_score = val_acc_faster_class * 0.5 + val_iou * 0.5
        scheduler.step(combined_score)

        xm.rendezvous("lr_scheduler_sync")

        with open("training_history.pkl", "wb") as f:
            pickle.dump(training_history, f)


        best_model_state = model.state_dict()

        if combined_score > best_combined_score:
            best_combined_score = combined_score
            xm.save(best_model_state, "best_model.pth")
            epochs_no_improve = 0
        else:
            epochs_no_improve += 1

        if epochs_no_improve >= patience_earlyStop:
            print("Early stopping activated.")
            break

        # Syncronize at the end of the epoch
        xm.mark_step()

    # Final cleaning
    xm.rendezvous("cleanup")

    # Print metrics 
    metrics = met.metrics_report()
    print(metrics)

    return training_history

## TRAIN

In [ ]:
model = CustomFaster()
model.load_state_dict(torch.load("model_path.pth"))

training_history = trainAndValid(model, train_dataloader, val_dataloader)